#### Fill in the blank to complete the following code and run:

In [1]:
import io
import base64
import numpy as np
from PIL import Image
import tensorflow as tf
import json
from flask import Flask, render_template, request
import requests

In [2]:
app = Flask(__name__, template_folder="./template")

img_height = 224
img_width = 224

class_names = {0: 'drum', 1: 'guitar', 2: 'piano', 3: 'violin'}

def loadmodel():    
    with open("./model/InceptionV3Adam.json","r") as json_file:
        loaded_model_json= json_file.read()
        
    #Load structure of model from json
    loaded_model= tf.keras.models.model_from_json(loaded_model_json)    
    #Load weight from model.h5 file
    loaded_model.load_weights("./model/InceptionV3Adam_weights.h5")
    
    return loaded_model

def prepare_image(img, img_url=None):
    if img_url:
        response = requests.get(img_url)
        img = Image.open(io.BytesIO(response.content))
    else:
        img = Image.open(io.BytesIO(img))
        
    img = img.convert("RGB")
    img = img.resize((img_height, img_width)) 
    img = np.array(img)
    img = img/255.0
    img = img.reshape(img_height, img_width, 3)   
    img = np.expand_dims(img, 0)
    return img

def predict_result(img):
    #Call load model
    model= loadmodel()
    rpredict= model.predict(img)

    return class_names[np.argmax(rpredict[0])]

@app.route("/")
def index():
    return render_template("homepage.html")

@app.route("/submiturl", methods=["GET", "POST"])
def submiturl():
    if request.method == "POST":
        url = request.form.get("img_url")
        if url:
            img = prepare_image(None, url)
            result = predict_result(img)
            return render_template("submiturl.html", result=result, img_url=url), 200
        else:
            return "Please try again. The image url is empty."
    else:
        # Render the template for GET requests
        return render_template("submiturl.html")
    
@app.route("/submitimage", methods=["GET", "POST"])
def submitimage():
    if request.method == "POST":
        if "imagefile" not in request.files:
            return "Please try again. The image does not exist."
        
        # Get the image file from request.files
        file = request.files["imagefile"]

        if not file:
            return "Please try again. No image file received."
        
          # Read image into bytes
        img_bytes = file.read()
        
        # Convert image bytes into base64 string
        img_string = base64.b64encode(img_bytes).decode()
        
        # Prepare image data
        img = prepare_image(img_bytes)
        
        # Classify the image data
        result = predict_result(img)

        return render_template("submitimage.html", img_data=img_string, prediction=result), 200
    else:
        # Render the template for GET requests
        return render_template("submitimage.html")  

if __name__ == "__main__":
    from werkzeug.serving import run_simple
    app.debug= True    
    run_simple("localhost", 5009, app)

 * Running on http://localhost:5009
Press CTRL+C to quit
127.0.0.1 - - [05/Feb/2023 19:03:12] "GET /submiturl HTTP/1.1" 200 -
127.0.0.1 - - [05/Feb/2023 19:03:21] "POST /submiturl HTTP/1.1" 200 -
127.0.0.1 - - [05/Feb/2023 19:04:11] "POST /submiturl HTTP/1.1" 200 -
127.0.0.1 - - [05/Feb/2023 19:04:18] "POST /submiturl HTTP/1.1" 200 -
